<a href="https://colab.research.google.com/github/aripo99/Simple_Networks/blob/main/nn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [80]:
cifar10 = datasets.CIFAR10("./", train=True, download=True,\
                    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4), # mean
                                                                                              (0.2470, 0.2435, 0.2616))])) # std
cifar10_val = datasets.CIFAR10("./", train=False, download=True,\
                    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4), 
                                                                                              (0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [81]:
print(len(cifar10), len(cifar10_val))

50000 10000


In [ ]:
img, label = cifar10[99]
print(img, label)

In [83]:
batch_size = 1
cifar10_train = DataLoader(cifar10, batch_size=batch_size)
cifar10_test = DataLoader(cifar10_val)

In [84]:
for batch in cifar10_train:
    print(batch[0].shape, batch[1].shape, batch[1])
    break

torch.Size([1, 3, 32, 32]) torch.Size([1]) tensor([6])


In [93]:
# Neural Network using Sequential API
network = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 10)
)

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.input = nn.Linear(3072, 512)
        self.output = nn.Linear(512, 10) # 10 classes
    
    def forward(self, x):
        x = torch.relu(self.input(x))
        x = self.output(x)
        return x

In [91]:
EPOCHS = 2
criterion = nn.CrossEntropyLoss()
# network = Network().to(device)
network.to(device) # using sequential
optimizer = optim.SGD(network.parameters(), lr=0.01) 

network.train()

loss_in_epoch = []
for epoch in range(EPOCHS):
    for batch in cifar10_train:
        optimizer.zero_grad()
        image = batch[0].to(device)
        label = batch[1].to(device)

        out = network(image.view(-1).unsqueeze(0))
        loss = criterion(out, torch.tensor([label]))

        loss.backward()
        optimizer.step()
    
    loss_in_epoch.append(loss.item())
    print("Epoch: {} \n Loss: {}".format(epoch, loss.item()))


Epoch: 0 
 Loss: 3.036949872970581
Epoch: 1 
 Loss: 4.406350612640381


In [92]:
correct = 0
total = 0

with torch.no_grad():
    for img, label in cifar10_test:
        outputs = network(img.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += label.shape[0]
        correct += int((predicted == label).sum())

print("Accuracy: {}".format(correct/total))

Accuracy: 0.2078
